# EVENT PLOTS for Krypton

A NB template to produce event plots (GUI version)

In [1]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

2017-04-05 16:54:24


In [2]:
%gui qt
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
import time
import tables as tb
import numpy as np

In [4]:
from invisible_cities.database import load_db

In [5]:
import invisible_cities.reco.tbl_functions as tbl
import invisible_cities.reco.wfm_functions as wfm
import invisible_cities.sierpe.blr as blr
import invisible_cities.reco.peak_functions_c as cpf
from invisible_cities.core.mpl_functions import set_plot_labels
from invisible_cities.reco.params import S12Params, ThresholdParams
from   invisible_cities.core.system_of_units_c import units

In [6]:
from gui_functions import fplot_pmt_waveforms, fplot_pmt_signals_vs_time_mus, fplot_signal_vs_time_mus,\
                           fplot_s12, hist_1d

In [7]:
from event_pmaps import EventPmaps, print_s12, print_s2si

In [8]:
run_number = 3389

In [9]:
DataPMT = load_db.DataPMT(run_number)

In [10]:
DataPMT

,SensorID,ChannelID,PmtID,Active,X,Y,coeff_blr,coeff_c,adc_to_pes,noise_rms,Sigma
0,0,22,PMT1,1,-23.9414,65.7785,0.00080,0.000001,25.39,0.7160,9.474
1,1,18,PMT3,1,-44.9951,-53.6231,0.00080,0.000001,26.49,0.8744,10.510
2,2,23,PMT2,1,68.9365,-12.1554,0.00080,0.000001,27.74,0.7303,9.031
3,3,0,PMT4,1,0.0000,185.0000,0.00161,0.000003,25.17,0.7060,8.720
4,4,31,PMTC,1,-118.9160,141.7180,0.00160,0.000003,26.56,0.7000,8.910
5,5,8,PMTB,1,-182.1890,32.1249,0.00161,0.000003,21.55,0.7278,6.900
6,6,5,PMTA,1,-160.2150,-92.5000,0.00161,0.000003,23.86,0.7323,7.860
7,7,4,PMT9,1,-63.2737,-173.8430,0.00161,0.000003,33.57,0.7206,11.750
8,8,1,PMT8,1,63.2737,-173.8430,0.00162,0.000003,22.15,0.7213,7.070
9,10,27,PMT6,1,182.1890,32.1249,0.00080,0.000001,20.83,0.7921,6.780


In [11]:
from PyQt5 import QtCore, QtWidgets, uic
from matplotlib.backends.backend_qt5agg import (
    FigureCanvasQTAgg as FigureCanvas,
    NavigationToolbar2QT as NavigationToolbar)
from matplotlib.figure import Figure

from PyQt5.uic import loadUiType


qtCreatorFile = "event_maps.ui" # Enter file here.
Ui_MainWindow, QMainWindow = loadUiType(qtCreatorFile)

In [12]:
class QtGui(QMainWindow, Ui_MainWindow):
    def __init__(self, ):
        super(QtGui, self).__init__()
        self.setupUi(self)
        
        self.fig_dict = {}
        self.mplfigs.itemClicked.connect(self.change_figure)

        fig = Figure()
        self.add_mpl(fig)

    def add_figure(self, name, fig):
        test_key = self.fig_dict.pop(name, None)
        self.fig_dict[name] = fig
        if not test_key: # key not in dict
            self.mplfigs.addItem(name)

    def change_figure(self, item):
        text = item.text()
        self.rm_mpl()
        self.add_mpl(self.fig_dict[text])

    def add_mpl(self, fig):
        self.canvas = FigureCanvas(fig)
        self.mplvl.addWidget(self.canvas)
        self.canvas.draw()
        self.toolbar = NavigationToolbar(self.canvas,
                self.mplwindow, coordinates=True)
        self.mplvl.addWidget(self.toolbar)

    def rm_mpl(self,):
        self.mplvl.removeWidget(self.canvas)
        self.canvas.close()
        self.mplvl.removeWidget(self.toolbar)
        self.toolbar.close()



In [13]:
def scan_events(epm, gui, pmtrwf, sipmrwf, 
                event_list=list(range(1)), 
                event_plot_list=list(range(1)), 
                printout=10):
    """Loop over events"""
    
    for event in event_list:
        if(event%printout == 0):
            print('processing event number {}'.format(event))
        epm.calibrated_pmt_and_csum(event, pmtrwf)
        epm.calibrated_sipm(event, sipmrwf)
        epm.find_s1()
        epm.find_s2()
        if (len(epm.S2) > 0):
            epm.find_s2si()
        
        if event in event_plot_list:
            #gui.add_figure('RWF-event-{}'.format(event), 
            #                plot_pmt_waveforms(epm.RWF, zoom=False, window_size=10000))
            gui.add_figure('CWF_vs_time_mus-event-{}'.format(event), 
                            fplot_pmt_signals_vs_time_mus(epm.CWF,
                                                         epm.P.pmt_active,
                                                         t_min      =    0,
                                                         t_max      = 1300,
                                                         signal_min =    -5,
                                                         signal_max =  200))
            gui.add_figure('Calibrated_SUM-event-{}'.format(event), 
                            fplot_signal_vs_time_mus(epm.csum, t_min=0, t_max=1300, signal_min=-5, signal_max=100))
            if (len(epm.S1) > 0):
                gui.add_figure('S1-event-{}'.format(event), fplot_s12(epm.S1))
            if (len(epm.S2) > 0):
                gui.add_figure('S2-event-{}'.format(event), fplot_s12(epm.S2))
                

In [14]:
RWF_file = os.path.join(os.environ['IC_DATA'],'LSC/wvfms/3389/dst_waves.gdcsnext.000_3389.root.h5')


In [15]:
h5rwf = tb.open_file(RWF_file,'r')

In [16]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [17]:
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))


        Number of events in file = 163
        Number of PMTs = 12
        PMTWL = 52000
        Number of SiPMs = 1792
        SiPMWL = 1300
      


In [18]:
s1par  = S12Params(tmin=0*units.mus, tmax=640*units.mus, lmin=5, lmax=20, stride=4, rebin=False)
s2par = S12Params(tmin=640*units.mus, tmax=800*units.mus, stride=40, lmin=80, lmax=20000, rebin=True)
thr    = ThresholdParams(thr_s1=0.5 * units.pes,  thr_s2=1 *units.pes,
                        thr_MAU = 3 * units.adc, thr_sipm = 3.5 * units.pes,
                        thr_SIPM = 30 * units.adc)

In [19]:
gui=QtGui()
gui.show()

In [20]:
epm = EventPmaps(run_number, s1par, s2par, thr, verbose=True)

In [21]:
event=0
epm.calibrated_pmt_and_csum(event, pmtrwf)

7477.3421116275758

In [22]:
gui.add_figure('CWF-event-{}'.format(event), 
                fplot_pmt_signals_vs_time_mus(epm.CWF,
                                             epm.P.pmt_active,
                                             t_min      =    0,
                                             t_max      = 1300,
                                             signal_min =    -5,
                                             signal_max =  250))

In [23]:
gui.add_figure('PMTs-event-{}'.format(event), 
                fplot_pmt_signals_vs_time_mus(epm.CAL_PMT,
                                             epm.P.pmt_active,
                                             t_min      =    0,
                                             t_max      = 1300,
                                             signal_min =    -5,
                                             signal_max =  10))

In [ ]:
n_events = 160
sample_plot = 10
t0 = time.time()
scan_events(epm, gui, pmtrwf, sipmrwf, 
            event_list=list(range(n_events)), 
            event_plot_list=list(range(0,n_events,sample_plot)),
           printout=10)
t1 = time.time()        
dt = t1 - t0
print("run {} events in {} s event/s = {}".format(n_events, dt, dt/n_events))

processing event number 0
number of peaks = 1
S12 number = 0, samples = 17 sum in pes =58.61823131191956
time vector (mus) = [ 381.675  381.7    381.725  381.75   381.775  381.8    381.825  381.85
  381.875  381.9    381.925  381.95   381.975  382.     382.075  382.1
  382.125]
energy vector (pes) = [ 1.61917637  4.26629217  7.76816226  9.1865877   8.86180842  7.24039865
  5.02407006  3.78509372  2.95172416  1.79096222  1.56561245  1.40410572
  0.902514    0.53159509  0.54448564  0.67117998  0.50446272]
number of peaks = 1
S12 number = 0, samples = 12 sum in pes =7002.0556670851765
time vector (mus) = [ 648.611875    649.6875      650.6875      651.6875      652.6875
  653.6875      654.6875      655.6875      656.6875      657.6875
  658.6925      659.72142857]
energy vector (pes) = [   92.34185897   305.01491887   609.24077745   986.05036791  1146.15608197
  1197.89848084  1079.29428206   699.55456278   539.60781327   262.37786222
    76.06448758     8.45417316]
S2Si for peak number 